# Wavelength Loss dependence using BraggExact

Here for the N2 configuration from Bird.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from scipy.optimize import newton

# Initialize

In [ ]:
n = 300
# wls = np.linspace(.4e-6, 1.6e-6, n+1)

wlLL = np.linspace(.4e-6, .48e-6, 40)[:-1]
wlL = np.linspace(.48e-6, .84e-6, 35)
wlC = np.linspace(.84e-6, .87e-6, 70)[1:-1]
wlR = np.linspace(.87e-6, 1.6e-6, 40)

wls = np.concatenate([wlLL, wlL, wlC, wlR])

betas1 = np.zeros_like(wls, dtype=complex)
outer = 'h2'
nu = 1

In [ ]:
# n_air = 1.00027717
# n_glass = 1.4388164768221814
# ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass]
# ts = [15*1e-6, .42*1e-6, 15*1e-6, 15*1e-6]

# 6 tube fiber

In [ ]:
ts = [15*1e-6, .42*1e-6, .832*15*1e-6, 10*1e-6]
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass]
mats = ['air', 'glass', 'air', 'glass']
maxhs = [100 ,100, 100, 100]
scale = 15*1e-6

In [ ]:
attempt_max = 25
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(len(wls)) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs, wl=wl, no_mesh=True, scale=scale)

    k_low = A.ks[0] * A.scale
    if wl <.8e-6:
        guess = np.array(.99995 * k_low)
    else:
        guess = np.array(.9999 * k_low)
    imag = 0
    flag = True
    reduce = 0
    attempts = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            if attempts >= attempt_max:
                beta = np.nan
                flag=False
            else:
                guess *= .999999
                reduce += 1
                print("scaling guess: " + str(reduce), flush=True)
                attempts += 1

    betas1[i] = beta
    

In [ ]:
# Formula for loss spikes from article

ms1 = np.arange(1,3,1)

n1 = A.ns[0]  # Inner (core) index
n2 = A.ns[1]  # Cladding index

d1 = ts[1]

ls1 = (2 * n1 * d1 / ms1 * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver


In [ ]:
%matplotlib notebook
plt.figure(figsize=(14,7))

CL = -20 * betas1.imag/A.scale / np.log(10)
msk = np.where(CL>0)
plt.plot(wls[msk], CL[msk], color='green', marker='o',linewidth=.9)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.yscale('log')
m, M = plt.ylim()
for l in ls1:
    plt.plot([l,l], [m, M], color='red', linewidth=1, linestyle='--')


Save data for pgfplots

In [ ]:
import os
paper_path = os.path.relpath(os.path.expanduser('~/papers/outer_materials/\
figures/data/bragg/6tube'))

both = np.column_stack((wls[msk]*1e6, CL[msk]))
np.savetxt(paper_path + '/analytic.dat', both, fmt='%.8f')

# 8 tube fiber


In [ ]:
n = 300
wls = np.linspace(2.4e-6, 13.6e-6, n+1)

# wlLL = np.linspace(.4e-6, .48e-6, 40)[:-1]
# wlL = np.linspace(.48e-6, .84e-6, 35)
# wlC = np.linspace(.84e-6, .87e-6, 70)[1:-1]
# wlR = np.linspace(.87e-6, 1.6e-6, 40)

# wls = np.concatenate([wlLL, wlL, wlC, wlR])

betas2 = np.zeros_like(wls, dtype=complex)
outer = 'h2'
nu = 1

In [ ]:
ts = [59.5*1e-6, 6*1e-6, .832*59.5*1e-6, 10*1e-6]
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass]
mats = ['air', 'glass', 'air', 'glass']
maxhs = [100 ,100, 100, 100]

In [ ]:
attempt_max = 25
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(len(wls)) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs, wl=wl, no_mesh=True)

    k_low = A.ks[0] * A.scale
    if wl <.8e-6:
        guess = np.array(.99995 * k_low)
    else:
        guess = np.array(.9999 * k_low)
    imag = 0
    flag = True
    reduce = 0
    attempts = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            if attempts >= attempt_max:
                beta = np.nan
                flag=False
            else:
                guess *= .999999
                reduce += 1
                print("scaling guess: " + str(reduce), flush=True)
                attempts += 1

    betas2[i] = beta
    

In [ ]:
# Formula for loss spikes from article

ms1 = np.arange(1,6,1)

n1 = A.ns[0]  # Inner (core) index
n2 = A.ns[1]  # Cladding index

d1 = ts[1]

ls1 = (2 * n1 * d1 / ms1 * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver


In [ ]:
%matplotlib notebook
plt.figure(figsize=(14,7))

CL2 = -20 * betas2.imag/A.scale / np.log(10)
msk = np.where(CL2>0)
plt.plot(wls[msk], CL2[msk], color='green', linewidth=.9)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.yscale('log')
m, M = plt.ylim()
for l in ls1:
    plt.plot([l,l], [m, M], color='red', linewidth=1, linestyle='--')


Save data for pgfplots

In [ ]:
import os
paper_path = os.path.relpath(os.path.expanduser('~/papers/outer_materials/\
figures/data/bragg/8tube'))

both = np.column_stack((wls[msk]*1e6, CL2[msk]))
np.savetxt(paper_path + '/analytic.dat', both, fmt='%.8f')